In [2]:
# 数据预处理
import pandas as pd

In [3]:
# 读取item part1
file_path1 = './Retailrocket recommender system dataset/item_properties_part1.csv'

data1 = pd.read_csv(file_path1)

print("数据预览：")
print(data1.head())

数据预览：
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566
3  1431226800000   59481         790                       n15360.000
4  1431831600000  156781         917                           828513


In [4]:
# item part2
file_path2 = './Retailrocket recommender system dataset/item_properties_part2.csv'

data2 = pd.read_csv(file_path2)

print("数据预览：")
print(data2.head())

数据预览：
       timestamp  itemid property            value
0  1433041200000  183478      561           769062
1  1439694000000  132256      976  n26.400 1135780
2  1435460400000  420307      921  1149317 1257525
3  1431831600000  403324      917          1204143
4  1435460400000  230701      521           769062


In [5]:
# events
event_path = './Retailrocket recommender system dataset/events.csv'

data_event = pd.read_csv(event_path)

print("数据预览：")
print(data_event.head())

数据预览：
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN


In [6]:
# category tree
category_path = './Retailrocket recommender system dataset/category_tree.csv'
data_category = pd.read_csv(category_path)
print("数据预览：")
print(data_category.head())

数据预览：
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0
3        1691     885.0
4         536    1691.0


In [7]:
event_rows = len(data_event)
print(f"events 总行数：{event_rows}")

category_rows = len(data_category)
print(f"category 总行数：{category_rows}")

part1_rows = len(data1)
part2_rows = len(data2)
total_rows = part1_rows + part2_rows
print(f"item_properties 总行数：{total_rows}")

events 总行数：2756101
category 总行数：1669
item_properties 总行数：20275902


In [8]:
if 'event' in data_event.columns:
    property_counts = data_event['event'].value_counts()
    
    print("\n 'event' 列的唯一值及其出现次数：")
    print(property_counts)
    
    print(f"\n总共有{property_counts.nunique()}种属性值")
    
else:
    print('none')


 'event' 列的唯一值及其出现次数：
event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

总共有3种属性值


In [9]:
if 'property' in data1.columns:
    property_counts = data1['property'].value_counts()
    
    print("\n 'property' 列的唯一值及其出现次数：")
    print(property_counts)
    
    print(f"\n总共有{property_counts.nunique()}种属性值")
    
else:
    print('none')


 'property' 列的唯一值及其出现次数：
property
888           1629817
790            970800
available      817387
categoryid     426305
6              343207
               ...   
782                 1
288                 1
427                 1
471                 1
712                 1
Name: count, Length: 1097, dtype: int64

总共有788种属性值


In [10]:
if 'property' in data2.columns:
    property_counts = data2['property'].value_counts()
    
    print("\n 'property' 列的唯一值及其出现次数：")
    print(property_counts)
    
    print(f"\n总共有{property_counts.nunique()}种属性值")
    
else:
    print('none')


 'property' 列的唯一值及其出现次数：
property
888           1370581
790            819716
available      686252
categoryid     361909
6              288264
               ...   
634                 1
472                 1
530                 1
703                 1
769                 1
Name: count, Length: 1094, dtype: int64

总共有760种属性值


从上面的预览中发现，item properties 里面的 property 和 value 都是十分奇怪的部分，经过查阅相关资料，property 内的绝大多数值都经过哈希处理从而变成数字，这对我们的数据仓库来说是很不友好的。但是不幸的是哈希加密是不可逆过程，我们没有办法找到原来的值是什么，所以考虑留下一些出现频率较高，删除频率小或将频率小的属性值归为 Others 。

In [11]:
n=300
top_properties1 = data1['property'].value_counts().head(n)

print(f"\n总数量前{n}的特征量为{top_properties1}")


总数量前300的特征量为property
888           1629817
790            970800
available      817387
categoryid     426305
6              343207
               ...   
411              2297
185              2295
730              2289
296              2275
997              2273
Name: count, Length: 300, dtype: int64


In [12]:
top_properties2 = data2['property'].value_counts().head(400)

print(f"\n总数量前400的特征量为{top_properties2}")


总数量前400的特征量为property
888           1370581
790            819716
available      686252
categoryid     361909
6              288264
               ...   
729              1212
679              1199
175              1177
669              1166
76               1160
Name: count, Length: 400, dtype: int64


In [20]:
bottom_properties1 = data1['property'].value_counts(ascending=True).head(400)

print((f"\n总数量前400的特征量为{bottom_properties1}"))


总数量前400的特征量为property
712       1
494       1
471       1
782       1
1027      1
       ... 
251     214
504     216
876     216
706     216
610     221
Name: count, Length: 400, dtype: int64


选择合适的阈值，允许适度的信息丢失，但是又不能丢失太多，同时也大量减少属性数量。本项目中希望忽略占比维持在1%~5%，适用于一般分析需求，允许适度的信息丢失，适合大部分商业和行为数据分析，能够平衡复杂性和覆盖率。

In [13]:
data = pd.concat([data1, data2])

# 统计属性值出现频率
property_counts = data['property'].value_counts()

# 设置阈值
threshold = 5000  # 忽略低于 20,000 次的属性

# 统计低频属性
low_frequency_properties = property_counts[property_counts < threshold]

# 统计高频属性
high_frequency_properties = property_counts[property_counts >= threshold]

# 输出统计结果
total_data = len(data)
low_freq_total = low_frequency_properties.sum()
high_freq_total = high_frequency_properties.sum()

print(f"总数据量：{total_data}")
print(f"低于 {threshold} 次的属性数量：{len(low_frequency_properties)}")
print(f"低频属性占总数据比例：{low_freq_total / total_data:.4%}")
print(f"高于等于 {threshold} 次的属性数量：{len(high_frequency_properties)}")
print(f"高频属性占总数据比例：{high_freq_total / total_data:.4%}")

总数据量：20275902
低于 5000 次的属性数量：834
低频属性占总数据比例：3.9891%
高于等于 5000 次的属性数量：270
高频属性占总数据比例：96.0109%


综上所述，我们就选择5000作为阈值，低于5000的归类为Others。

在实际操作过程中，我们又发现了另外的问题。因为SSIS读取文件内存大小是有限的，但是我们的数据对于SSIS来说还算庞大的，在导入过程中要么报错要么就是减少了几百万条数据甚至一千多万数据。经过对教程数据库AdventureWorksDW2022的观察，我们发现这个数据库种建了很多同种类型的表，所以我们采取的做法是将数据分割成一个个小块文件，每个文件装入50000条数据，防止SSIS出现问题。

其实本来是可以直接修改数据流的默认配置，修改缓冲区大小和最大行数，正好又学习过Foreach循环，所以同时也顺便直接切割了。

In [44]:
output_path = "./databaseForSSIS/"
chunk_size = 50000

In [45]:
# events
for i, chunk in enumerate(pd.read_csv(event_path, chunksize=chunk_size)):
    chunk.to_csv(f"{output_path}events_part{i+1}.csv", index=False)
    print(f"生成文件：events_part{i+1}.csv")

生成文件：events_part1.csv
生成文件：events_part2.csv
生成文件：events_part3.csv
生成文件：events_part4.csv
生成文件：events_part5.csv
生成文件：events_part6.csv
生成文件：events_part7.csv
生成文件：events_part8.csv
生成文件：events_part9.csv
生成文件：events_part10.csv
生成文件：events_part11.csv
生成文件：events_part12.csv
生成文件：events_part13.csv
生成文件：events_part14.csv
生成文件：events_part15.csv
生成文件：events_part16.csv
生成文件：events_part17.csv
生成文件：events_part18.csv
生成文件：events_part19.csv
生成文件：events_part20.csv
生成文件：events_part21.csv
生成文件：events_part22.csv
生成文件：events_part23.csv
生成文件：events_part24.csv
生成文件：events_part25.csv
生成文件：events_part26.csv
生成文件：events_part27.csv
生成文件：events_part28.csv
生成文件：events_part29.csv
生成文件：events_part30.csv
生成文件：events_part31.csv
生成文件：events_part32.csv
生成文件：events_part33.csv
生成文件：events_part34.csv
生成文件：events_part35.csv
生成文件：events_part36.csv
生成文件：events_part37.csv
生成文件：events_part38.csv
生成文件：events_part39.csv
生成文件：events_part40.csv
生成文件：events_part41.csv
生成文件：events_part42.csv
生成文件：events_part43.csv
生成文件：events_part44.c

In [14]:
print("拼接part1和part2文件...")
combined = pd.concat([data1, data2], ignore_index=True)
print(f"拼接完成，总行数: {len(combined)}")

拼接part1和part2文件...
拼接完成，总行数: 20275902


In [39]:
# 统计 property 的出现次数
print("正在统计 property 的出现次数...")
property_counts = combined['property'].value_counts()

# 将低于阈值的 property 设置为 "Other"
print(f"将出现次数低于 {threshold} 次的 property 设置为 'Other'...")
combined['property'] = combined['property'].apply(
    lambda x: x if property_counts[x] >= threshold else 'Other'
)

正在统计 property 的出现次数...
将出现次数低于 5000 次的 property 设置为 'Other'...


In [46]:
# 分割为多个文件
print(f"开始分割文件，每个文件包含 {chunk_size} 行...")
for i in range(0, len(combined), chunk_size):
    output_file = f"{output_path}item_properties_part{i // chunk_size + 1}.csv"
    combined.iloc[i:i + chunk_size].to_csv(output_file, index=False)
    print(f"生成文件：{output_file}，行数：{len(combined.iloc[i:i + chunk_size])}")

print("文件分割完成！")

开始分割文件，每个文件包含 50000 行...
生成文件：./databaseForSSIS/item_properties_part1.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part2.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part3.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part4.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part5.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part6.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part7.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part8.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part9.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part10.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part11.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part12.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part13.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part14.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part15.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_part16.csv，行数：50000
生成文件：./databaseForSSIS/item_properties_p

In [17]:
import numpy as np

In [19]:
dim_property = combined[['property','value']].drop_duplicates().reset_index(drop=True)
dim_property = dim_property.reset_index()
dim_property.columns = ['PropertyID', 'PropertyName','Value']
dim_property['CategoryID'] = np.where(dim_property['PropertyName'] == 'categoryid', dim_property['Value'], np.nan)
dim_property = dim_property.drop(columns=['Value'])
dim_property.to_csv('./databaseForSSIS/dim_property.csv', index=False)
print("done")

done
